<a href="https://colab.research.google.com/github/jessilver/Organizacao_Computadores/blob/develop/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faça uma cópia no seu drive

Importante: as instalações precisam ser feita toda vez que for iniciada uma nova seção no Colab

# Jessé Eliseu Nunes da Silva

# Como usar:

As implementações foram separadas e partes, para que a interface reconheça a parte que desesja utilizar bastas "ativa-la", trocando o valor da variavel de status de false para true.

## Teste:

```
# --- Teste de Operações com Inteiros ---

# Teste DIV
ADDI x1, x0, 100
ADDI x2, x0, 10
DIV  x3, x1, x2  # x3 = 100 / 10 = 10

# Teste de Overflow na Soma (AGORA VAI GERAR O ERRO)
ADDI x5, x0, 2147483647
ADDI x6, x0, 1
ADD  x7, x5, x6  

# O código abaixo não será executado, pois o programa
# vai parar no erro de overflow acima.
# --- Teste de Operações com Ponto Flutuante ---
ADDI x10, x0, 1075838976
ADDI x11, x0, 1080033280
ADD.S x12, x10, x11
MUL.S x13, x10, x11
```

# Parte 1


In [44]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import struct

# ===================================================================
# 1. DEFINIÇÃO DOS WIDGETS DA INTERFACE
# ===================================================================
code_area = widgets.Textarea(
    value="",
    placeholder="Escreva seu código assembly aqui...",
    description="Code:",
    layout=widgets.Layout(width="100%", height="200px")
)
run_button = widgets.Button(description="Run Program", button_style="success")
reg_button = widgets.Button(description="Show Registers", button_style="info")
mem_button = widgets.Button(description="Show Memory", button_style="info")
start_addr = widgets.IntText(value=0, description="Start Addr")
mem_range = widgets.IntText(value=10, description="Range")
output_area = widgets.Output()

# ===================================================================
# 2. FUNÇÕES AUXILIARES E DE EXIBIÇÃO
# ===================================================================

# --- Funções Auxiliares para Ponto Flutuante (Requisito da Parte 1) ---
def float_to_int_bits(f):
    return struct.unpack('I', struct.pack('f', float(f)))[0]

def int_bits_to_float(i):
    try:
        return struct.unpack('f', struct.pack('I', i & 0xFFFFFFFF))[0]
    except (struct.error, ValueError):
        return float('nan')

def to_signed32(value):
    value = value & 0xFFFFFFFF
    if value & (1 << 31):
        return value - (1 << 32)
    return value

# --- Funções de Exibição ---
def show_registers(regs):
    """
    Exibe os registradores. A coluna "Valor FP" foi adicionada
    para facilitar a verificação dos requisitos da Parte 1.
    """
    output_widgets = []
    for i in range(0, 32, 8):
        start_reg, end_reg = i, i + 8
        register_values = [regs.read(j) for j in range(start_reg, end_reg)]
        data = {
            "Reg": [f"x{j}" for j in range(start_reg, end_reg)],
            "Valor Int": [to_signed32(val) for val in register_values],
            # Coluna extra para a Parte 1
            "Valor FP": [f"{int_bits_to_float(val):.4f}" for val in register_values]
        }
        df = pd.DataFrame(data).set_index("Reg")
        out = widgets.Output()
        with out: display(df)
        output_widgets.append(out)
    with output_area:
        clear_output(wait=True)
        display(widgets.HBox(output_widgets))

def show_memory(mem, start=0, rng=10):
    try:
        data = {"Endereço": list(range(start, start + rng)), "Valor": [mem.load_word(i) for i in range(start, start + rng)]}
        df = pd.DataFrame(data)
        with output_area:
            clear_output(wait=True)
            display(df)
    except IndexError as e:
        with output_area:
            clear_output(wait=True)
            print(f"Erro ao acessar memória: {e}")

# ===================================================================
# 3. CLASSES DO SIMULADOR (MEMÓRIA E REGISTRADORES)
# (Estrutura base para ambas as partes)
# ===================================================================
class SimpleMemory:
    def __init__(self, size_in_words):
        self.size = size_in_words
        self.mem = [0] * size_in_words

    def load_word(self, address):
        if 0 <= address < self.size:
            return self.mem[address]
        raise IndexError(f"Endereço de memória inválido: {address}")

    def store_word(self, address, value):
        if 0 <= address < self.size:
            self.mem[address] = value & 0xFFFFFFFF
        else:
            raise IndexError(f"Endereço de memória inválido: {address}")

class RegisterFile:
    def __init__(self):
        self.regs = [0] * 32

    def read(self, reg_num):
        if 0 <= reg_num < 32:
            return self.regs[reg_num]
        raise IndexError(f"Número de registrador inválido: x{reg_num}")

    def write(self, reg_num, value):
        if reg_num == 0: return
        if 0 < reg_num < 32:
            self.regs[reg_num] = value & 0xFFFFFFFF
        else:
            raise IndexError(f"Número de registrador inválido: x{reg_num}")

# ===================================================================
# 4. LÓGICA PRINCIPAL DO SIMULADOR (PARSE E EXECUTE)
# ===================================================================
def execute_instruction(instr, regs, mem):
    op = instr["op"]
    info = ""
    INT32_MIN, INT32_MAX = -2**31, 2**31 - 1

    # --- Bloco de Operações Aritméticas ---
    if op in ["ADD", "ADDI", "SUB", "MUL", "DIV"]:
        # Parte 2: Implementa ADD, SUB, MUL.
        # Parte 1: Implementa DIV e a detecção de OVERFLOW para todas as operações.
        rs1_val = to_signed32(regs.read(instr["rs1"]))
        rs2_val = instr["imm"] if op == "ADDI" else to_signed32(regs.read(instr["rs2"]))
        op_symbol = {'ADD': '+', 'ADDI': '+', 'SUB': '-', 'MUL': '*', 'DIV': '/'}[op]

        if op in ["ADD", "ADDI"]: result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        elif op == "MUL": result = rs1_val * rs2_val
        elif op == "DIV": # Requisito da Parte 1
            if rs2_val == 0: raise ValueError("Divisão por zero")
            if rs1_val == INT32_MIN and rs2_val == -1: raise OverflowError(f"Overflow na divisão: {rs1_val} / {rs2_val}")
            result = int(rs1_val / rs2_val)

        # Detecção de Overflow (Requisito da Parte 1)
        if not (INT32_MIN <= result <= INT32_MAX) and op != "DIV":
            raise OverflowError(f"Overflow na operação: {rs1_val} {op_symbol} {rs2_val}")

        regs.write(instr["rd"], result)
        info = f"x{instr['rd']} = {rs1_val} {op_symbol} {rs2_val} -> {result}"

    # --- Bloco de Ponto Flutuante (Requisito da Parte 1) ---
    elif op in ["ADD.S", "MUL.S"]:
        float1 = int_bits_to_float(regs.read(instr["rs1"]))
        float2 = int_bits_to_float(regs.read(instr["rs2"]))
        op_symbol = '+' if op == "ADD.S" else '*'
        result_float = float1 + float2 if op == "ADD.S" else float1 * float2
        result_bits = float_to_int_bits(result_float)
        regs.write(instr["rd"], result_bits)
        info = f"x{instr['rd']} = {float1:.2f} {op_symbol} {float2:.2f} -> {result_float:.2f}"

    # --- Bloco de Acesso à Memória (Requisito da Parte 2) ---
    elif op == "LW": # LOAD
        addr, rd = instr["addr"], instr["rd"]
        value = mem.load_word(addr)
        regs.write(rd, value)
        info = f"x{rd} = mem[{addr}] (Valor: {value})"

    elif op == "SW": # STORE
        addr, rs1 = instr["addr"], instr["rs1"]
        value = regs.read(rs1)
        mem.store_word(addr, value)
        info = f"mem[{addr}] = x{rs1} (Valor: {value})"

    else:
        info = f"Instrução desconhecida {op}"

    return info

def parse_program(asm_code):
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0]
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()

        # Parte 2: Reconhece ADD, SUB, MUL
        # Parte 1: Reconhece DIV, ADD.S, MUL.S
        if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]:
            instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "ADDI": # Instrução extra (Parte 1)
            instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "imm": int(parts[3])}
        elif op == "LW" or op == "SW": # Parte 2: Reconhece LOAD (LW) e STORE (SW)
            if op == "LW":
                instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
            else: # SW
                instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else:
            raise ValueError(f"Instrução desconhecida: {line}")
        program.append(instr)
    return program

def run_program(asm_code, regs, mem):
    # Esta função implementa o Datapath de Ciclo Único (Requisito da Parte 2).
    # O loop 'for' executa uma instrução por vez, completamente, antes de
    # passar para a próxima, o que caracteriza a ausência de pipeline.
    try:
        program = parse_program(asm_code)
    except ValueError as e:
        print(f"ERRO DE SINTAXE NO CÓDIGO: {e}")
        return

    for cycle, instr in enumerate(program, 1):
        formatted_instr = format_instruction(instr)
        try:
            info = execute_instruction(instr, regs, mem)
            print(f"Ciclo {cycle}: {formatted_instr:<25} -> {info}")
        except (ValueError, IndexError, OverflowError) as e:
            print(f"Ciclo {cycle}: {formatted_instr:<25} -> ERRO: {e}")

def format_instruction(instr):
    op = instr['op']
    if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]:
        return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, x{instr['rs2']}"
    elif op == "ADDI":
        return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, {instr['imm']}"
    elif op == "LW":
        return f"{op:<8} x{instr['rd']}, {instr['addr']}"
    elif op == "SW":
        return f"{op:<8} x{instr['rs1']}, {instr['addr']}"
    return str(instr)

# ===================================================================
# 5. HANDLERS E CONFIGURAÇÃO DA INTERFACE
# ===================================================================
def run_program_handler(b):
    with output_area:
        clear_output(wait=True)
        print("Inicializando registradores e memória...")
    global regs, mem
    regs = RegisterFile()
    mem = SimpleMemory(64)
    code = code_area.value
    print("--- Log de Execução ---")
    run_program(code, regs, mem)
    print("--- Fim da Execução ---")
    with output_area:
        print("Programa executado. Use os botões para inspecionar o estado.")

def reg_button_handler(b):
    show_registers(regs)

def mem_button_handler(b):
    show_memory(mem, start=start_addr.value, rng=mem_range.value)

run_button.on_click(run_program_handler)
reg_button.on_click(reg_button_handler)
mem_button.on_click(mem_button_handler)

regs, mem = RegisterFile(), SimpleMemory(64)
ui = widgets.VBox([
    code_area,
    widgets.HBox([run_button, reg_button, mem_button, start_addr, mem_range]),
    output_area
])
display(ui)

--- Log de Execução ---
Ciclo 1: ADDI     x1, x0, 100      -> x1 = 0 + 100 -> 100
Ciclo 2: ADDI     x2, x0, 10       -> x2 = 0 + 10 -> 10
Ciclo 3: DIV      x3, x1, x2       -> x3 = 100 / 10 -> 10
Ciclo 4: ADDI     x5, x0, 2147483647 -> x5 = 0 + 2147483647 -> 2147483647
Ciclo 5: ADDI     x6, x0, 1        -> x6 = 0 + 1 -> 1
Ciclo 6: ADD      x7, x5, x6       -> ERRO: Overflow na operação: 2147483647 + 1
Ciclo 7: ADDI     x10, x0, 1075838976 -> x10 = 0 + 1075838976 -> 1075838976
Ciclo 8: ADDI     x11, x0, 1080033280 -> x11 = 0 + 1080033280 -> 1080033280
Ciclo 9: ADD.S    x12, x10, x11    -> x12 = 2.50 + 3.50 -> 6.00
Ciclo 10: MUL.S    x13, x10, x11    -> x13 = 2.50 * 3.50 -> 8.75
--- Fim da Execução ---


# Parte 2

# Parte 3

# Parte 4

# Parte 5